# Importing Relevant Modules

In [23]:
import pandas as pd
import json
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.preprocessing import MinMaxScaler
from functools import reduce
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)

# Read Data from Input & Pre-processing

In [2]:
xls = pd.ExcelFile('sample data.xlsx')

farmer_info = pd.read_excel(xls, 'Farmer Info')
# crop_cycle = pd.read_excel(xls, 'Crop Cycle')
loan_status = pd.read_excel(xls, 'Loan Status')
business_profile = pd.read_excel(xls, 'Business Profile')
household_finance = pd.read_excel(xls, 'Household Finance')
personal_info_finance = pd.read_excel(xls, 'Personal Information-Finance')
personal_info_me = pd.read_excel(xls, 'Personal Information-ME')

In [3]:
df1 = farmer_info[['accountID', 'gender', 'last_education', 'housing_ownership', 'years_stay_residing', 'marital_status', 'number_of_children']].copy()
df1

,accountID,gender,last_education,housing_ownership,years_stay_residing,marital_status,number_of_children
0,10001,Male,Junior High School,One's own,50,Married,3
1,10002,Male,Junior High School,One's own,10,Married,1
2,10003,Female,Junior High School,One's own,8,Single,2


In [4]:
# df2 = crop_cycle[['land_area', 'plot_age', 'land_ownership_status', 'start_year', 'soil_quality', 'tree_no', 'produtive_trees_no','estimated_harvest_date', 'crop_type_code', 'estimated_yield_of_harvest', 'irrigation_system_code', 'is_submit', 'equipment_used']]
# df2

In [5]:
df3 = loan_status[['accountID', 'actual_loan_amount', 'loan_amount', 'repayment_amount', 'paid_amount','status_id']].copy()
df3

,accountID,actual_loan_amount,loan_amount,repayment_amount,paid_amount,status_id
0,10001,150000000,150000000,1.606409e+08,150000000,Servicing
1,10002,130000000,130000000,1.606409e+08,150000000,Completed
2,10003,200000000,200000000,1.606409e+08,150000000,Defaulted


In [6]:
def extract_value(x):
    return int(json.loads(x)['amount'])

df4 = business_profile[['accountID', 'years_in_business', 'number_of_employees', 'weekly_employee_salaries', 'weekly_turnover', 'weekly_working_capital', 'weekly_business_expense', 'weekly_net_profit', 'outstanding_debt']].copy()
df4

for col in df4.columns:
    if col not in ['accountID', 'years_in_business', 'number_of_employees']:
        df4[col] = df4[col].apply(lambda x: extract_value(x))
df4

,accountID,years_in_business,number_of_employees,weekly_employee_salaries,weekly_turnover,weekly_working_capital,weekly_business_expense,weekly_net_profit,outstanding_debt
0,10001,28,7,1750,2500000,1500000,1350000,1000000,0
1,10002,7,3,600,2000000,1000000,1000000,750000,0
2,10003,10,10,2500,1750000,800000,900000,400000,20000


In [7]:
df5 = household_finance.copy()

for col in df5.columns:
    if col not in ['accountID', 'dependent', 'family_health']:
        df5[col] = df5[col].apply(lambda x: extract_value(x))
        
df5

,accountID,earning_12month,earning_3month,dependent,weekly_expense,family_health,monthly_mortgage,monthly_credit_card,outstanding_credit,debt_90days_late
0,10001,24000000,6000000,2,0,Very Good,0,0,0,0
1,10002,15000000,3750000,1,0,Very Good,0,0,0,0
2,10003,5000000,1250000,0,600,Very Good,0,0,0,0


In [8]:
df6 = personal_info_finance.drop('bank_country', axis=1).copy()
df6
# Apply one-hot encoding

,accountID,type_of_income,declared_bankruptcy,currently_bankrupt_foreclosure,job_id,online_job,electronic_goods_guarantee,bank_name
0,10001,Fixed,no,no,entrepreneur,Not Online Based,TV,BANK NEGARA INDONESIA DEBIT PLUS
1,10002,Fixed,no,no,entrepreneur,Not Online Based,TV,BANK NEGARA INDONESIA DEBIT PLUS
2,10003,Fixed,no,no,entrepreneur,Not Online Based,TV,BANK NEGARA INDONESIA DEBIT PLUS


In [9]:
df7 = personal_info_me[['accountID','vehicle_ownership', 'vehicle_status']].copy()
df7
# Apply one-hot encoding

,accountID,vehicle_ownership,vehicle_status
0,10001,Motor,One's own
1,10002,Car,One's own
2,10003,Car,One's own


In [10]:
df = reduce(lambda left,right: pd.merge(left,right,on='accountID'), [df1, df3, df4, df5, df6, df7])
df

,accountID,gender,last_education,housing_ownership,years_stay_residing,marital_status,number_of_children,actual_loan_amount,loan_amount,repayment_amount,paid_amount,status_id,years_in_business,number_of_employees,weekly_employee_salaries,weekly_turnover,weekly_working_capital,weekly_business_expense,weekly_net_profit,outstanding_debt,earning_12month,earning_3month,dependent,weekly_expense,family_health,monthly_mortgage,monthly_credit_card,outstanding_credit,debt_90days_late,type_of_income,declared_bankruptcy,currently_bankrupt_foreclosure,job_id,online_job,electronic_goods_guarantee,bank_name,vehicle_ownership,vehicle_status
0,10001,Male,Junior High School,One's own,50,Married,3,150000000,150000000,1.606409e+08,150000000,Servicing,28,7,1750,2500000,1500000,1350000,1000000,0,24000000,6000000,2,0,Very Good,0,0,0,0,Fixed,no,no,entrepreneur,Not Online Based,TV,BANK NEGARA INDONESIA DEBIT PLUS,Motor,One's own
1,10002,Male,Junior High School,One's own,10,Married,1,130000000,130000000,1.606409e+08,150000000,Completed,7,3,600,2000000,1000000,1000000,750000,0,15000000,3750000,1,0,Very Good,0,0,0,0,Fixed,no,no,entrepreneur,Not Online Based,TV,BANK NEGARA INDONESIA DEBIT PLUS,Car,One's own
2,10003,Female,Junior High School,One's own,8,Single,2,200000000,200000000,1.606409e+08,150000000,Defaulted,10,10,2500,1750000,800000,900000,400000,20000,5000000,1250000,0,600,Very Good,0,0,0,0,Fixed,no,no,entrepreneur,Not Online Based,TV,BANK NEGARA INDONESIA DEBIT PLUS,Car,One's own


## Converting Categorical Features to Dummy Variabels

In [11]:
cols = df.columns
num_cols = df._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))

for col in cat_cols:
    if col != 'status_id':
        temp = pd.get_dummies(df[col])
        df = df.drop([col], axis=1)
        df = pd.concat([df, temp], axis=1)

## Scaling Numerical Features to the range between 0 and 1

In [12]:
scaler = MinMaxScaler()
num_cols = list(num_cols)

for col in num_cols:
    if col != 'accountID':
        df[col] = scaler.fit_transform(df[col].values.reshape(-1,1))

## Map Dependent Variable to Numeric Classes

In [15]:
def map(x):
    if x == 'Servicing':
        return 1
    elif x == 'Completed':
        return 2
    else:
        return 0

df['status_id'] = df['status_id'].apply(lambda x : map(x))

In [16]:
df

,accountID,years_stay_residing,number_of_children,actual_loan_amount,loan_amount,repayment_amount,paid_amount,status_id,years_in_business,number_of_employees,weekly_employee_salaries,weekly_turnover,weekly_working_capital,weekly_business_expense,weekly_net_profit,outstanding_debt,earning_12month,earning_3month,dependent,weekly_expense,monthly_mortgage,monthly_credit_card,outstanding_credit,debt_90days_late,Car,Motor,no,Female,Male,Fixed,BANK NEGARA INDONESIA DEBIT PLUS,no,Not Online Based,entrepreneur,TV,Married,Single,One's own,Very Good,One's own,Junior High School
0,10001,1.000000,1.0,0.285714,0.285714,0.0,0.0,1,1.000000,0.571429,0.605263,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1
1,10002,0.047619,0.0,0.000000,0.000000,0.0,0.0,2,0.000000,0.000000,0.000000,0.333333,0.285714,0.222222,0.583333,0.0,0.526316,0.526316,0.5,0.0,0.0,0.0,0.0,0.0,1,0,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1
2,10003,0.000000,0.5,1.000000,1.000000,1.0,0.0,0,0.142857,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1


# SVM

## Declaring Independent Variables X, Dependent Variable y

In [17]:
X = df.drop('status_id', axis= 1)
y = df['status_id']

## Training SVM Classifier

In [21]:
clf = svm.SVC(kernel = 'linear')
clf.fit(X, y)

SVC(kernel='linear')